# Anomaly Detection with Isolation Forest

Using IsolationForest to detect early hospital readmissions (<30 days) as anomalies.

## Setup and Data Loading

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, auc
from sklearn.metrics import precision_score, recall_score, classification_report

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 5)

print("Libraries loaded successfully!")

In [ ]:
# Load preprocessed data
# Assuming you have X and y from preprocessing
# If loading from CSV:
X = pd.read_csv('../data/processed/X_features.csv')
y = pd.read_csv('../data/processed/y_target.csv')['target'].values

print(f"Feature matrix shape: {X.shape}")
print(f"Target vector shape: {y.shape}")
print(f"\nClass distribution:")
print(f"  y=0 (normal, not readmitted <30): {(y==0).sum()} ({(y==0).sum()/len(y)*100:.1f}%)")
print(f"  y=1 (anomaly, readmitted <30):    {(y==1).sum()} ({(y==1).sum()/len(y)*100:.1f}%)")

## 1. Train/Test Split (Stratified)

In [ ]:
# Stratified split to preserve class distribution
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

print("Train/Test Split Complete")
print("=" * 50)
print(f"Train set: {X_train.shape[0]:,} samples")
print(f"  y=0: {(y_train==0).sum():,} ({(y_train==0).sum()/len(y_train)*100:.1f}%)")
print(f"  y=1: {(y_train==1).sum():,} ({(y_train==1).sum()/len(y_train)*100:.1f}%)")

print(f"\nTest set: {X_test.shape[0]:,} samples")
print(f"  y=0: {(y_test==0).sum():,} ({(y_test==0).sum()/len(y_test)*100:.1f}%)")
print(f"  y=1: {(y_test==1).sum():,} ({(y_test==1).sum()/len(y_test)*100:.1f}%)")

## 2. Train Isolation Forest (on Normal Class Only)

IsolationForest is an unsupervised method. We train it on the **majority class (y=0)** only, treating these as "normal" samples.

In [ ]:
# Get only normal samples from training set (y=0)
X_train_normal = X_train[y_train == 0]

print(f"Training IsolationForest on {len(X_train_normal):,} normal samples...")

# Initialize and fit IsolationForest
# contamination: expected proportion of outliers (set to match actual anomaly rate)
iso_forest = IsolationForest(
    n_estimators=100,
    max_samples='auto',
    contamination=0.11,  # Approximately 11% anomalies based on data
    random_state=42,
    n_jobs=-1,
    verbose=0
)

# Fit on normal samples only
iso_forest.fit(X_train_normal)

print("✓ IsolationForest trained successfully!")

## 3. Compute Anomaly Scores

IsolationForest returns:
- **decision_function**: Higher (less negative) = more normal, Lower (more negative) = more anomalous
- We'll convert to anomaly scores where **higher = more anomalous**

In [ ]:
# Get anomaly scores (decision function)
# Note: decision_function returns negative values where lower = more anomalous
# We'll negate it so higher scores = more anomalous
train_scores = -iso_forest.decision_function(X_train)
test_scores = -iso_forest.decision_function(X_test)

# Also get binary predictions (-1 for anomaly, 1 for normal)
train_predictions = iso_forest.predict(X_train)
test_predictions = iso_forest.predict(X_test)

print("Anomaly Scores Computed")
print("=" * 50)
print(f"Train scores - Min: {train_scores.min():.3f}, Max: {train_scores.max():.3f}, Mean: {train_scores.mean():.3f}")
print(f"Test scores  - Min: {test_scores.min():.3f}, Max: {test_scores.max():.3f}, Mean: {test_scores.mean():.3f}")

# Predicted anomalies (default threshold)
train_anomalies = (train_predictions == -1).sum()
test_anomalies = (test_predictions == -1).sum()
print(f"\nPredicted anomalies (default threshold):")
print(f"  Train: {train_anomalies:,} ({train_anomalies/len(y_train)*100:.1f}%)")
print(f"  Test:  {test_anomalies:,} ({test_anomalies/len(y_test)*100:.1f}%)")

## 4. Evaluation Metrics on Test Set

In [ ]:
# Calculate ROC-AUC
roc_auc = roc_auc_score(y_test, test_scores)

# Calculate Precision-Recall AUC
precision_vals, recall_vals, pr_thresholds = precision_recall_curve(y_test, test_scores)
pr_auc = auc(recall_vals, precision_vals)

print("Overall Metrics on Test Set")
print("=" * 50)
print(f"ROC-AUC:             {roc_auc:.4f}")
print(f"Precision-Recall AUC: {pr_auc:.4f}")

# Evaluate at different thresholds
print("\n" + "=" * 50)
print("Performance at Different Thresholds")
print("=" * 50)

# Define some percentile-based thresholds
thresholds_to_test = [
    np.percentile(test_scores, 90),  # Top 10% most anomalous
    np.percentile(test_scores, 85),  # Top 15% most anomalous
    np.percentile(test_scores, 80),  # Top 20% most anomalous
]

threshold_results = []

for threshold in thresholds_to_test:
    # Predict anomalies based on threshold
    y_pred = (test_scores >= threshold).astype(int)
    
    # Calculate metrics
    precision = precision_score(y_test, y_pred, zero_division=0)
    recall = recall_score(y_test, y_pred, zero_division=0)
    flagged_pct = (y_pred == 1).sum() / len(y_pred) * 100
    
    threshold_results.append({
        'Threshold': threshold,
        'Precision': precision,
        'Recall': recall,
        'Flagged (%)': flagged_pct
    })
    
    print(f"\nThreshold: {threshold:.3f}")
    print(f"  Precision: {precision:.3f}")
    print(f"  Recall:    {recall:.3f}")
    print(f"  Flagged:   {flagged_pct:.1f}% of test samples")

# Create summary DataFrame
threshold_df = pd.DataFrame(threshold_results)
print("\n" + "=" * 50)
print("Summary Table")
print("=" * 50)
print(threshold_df.to_string(index=False))

## 5. Visualizations

In [ ]:
# Calculate ROC curve
fpr, tpr, roc_thresholds = roc_curve(y_test, test_scores)

# Create figure with subplots
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ========== ROC Curve ==========
axes[0].plot(fpr, tpr, linewidth=2, label=f'IsolationForest (AUC = {roc_auc:.3f})')
axes[0].plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random Classifier')
axes[0].set_xlabel('False Positive Rate', fontsize=11)
axes[0].set_ylabel('True Positive Rate', fontsize=11)
axes[0].set_title('ROC Curve - Anomaly Detection', fontsize=12, fontweight='bold')
axes[0].legend(loc='lower right')
axes[0].grid(alpha=0.3)

# ========== Precision-Recall Curve ==========
axes[1].plot(recall_vals, precision_vals, linewidth=2, 
             label=f'IsolationForest (AUC = {pr_auc:.3f})')

# Baseline (random classifier for imbalanced data)
baseline_precision = (y_test == 1).sum() / len(y_test)
axes[1].axhline(y=baseline_precision, color='k', linestyle='--', linewidth=1, 
                label=f'Baseline (Random) = {baseline_precision:.3f}')

axes[1].set_xlabel('Recall', fontsize=11)
axes[1].set_ylabel('Precision', fontsize=11)
axes[1].set_title('Precision-Recall Curve', fontsize=12, fontweight='bold')
axes[1].legend(loc='best')
axes[1].grid(alpha=0.3)

plt.tight_layout()

# Save figure
from pathlib import Path
output_dir = Path('../results/figures')
output_dir.mkdir(parents=True, exist_ok=True)
plt.savefig(output_dir / 'isolation_forest_evaluation.png', dpi=150, bbox_inches='tight')
print(f"✓ Saved plot to: {output_dir / 'isolation_forest_evaluation.png'}")

plt.show()

## 6. Additional Analysis: Score Distribution by Class

In [ ]:
# Visualize anomaly score distributions for both classes
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(test_scores[y_test == 0], bins=50, alpha=0.6, label='Normal (y=0)', color='green')
axes[0].hist(test_scores[y_test == 1], bins=50, alpha=0.6, label='Anomaly (y=1)', color='red')
axes[0].set_xlabel('Anomaly Score', fontsize=11)
axes[0].set_ylabel('Frequency', fontsize=11)
axes[0].set_title('Distribution of Anomaly Scores by True Label', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Box plot
axes[1].boxplot([test_scores[y_test == 0], test_scores[y_test == 1]], 
                labels=['Normal (y=0)', 'Anomaly (y=1)'],
                patch_artist=True,
                boxprops=dict(facecolor='lightblue', alpha=0.7))
axes[1].set_ylabel('Anomaly Score', fontsize=11)
axes[1].set_title('Anomaly Score Distribution (Box Plot)', fontsize=12, fontweight='bold')
axes[1].grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(output_dir / 'anomaly_score_distributions.png', dpi=150, bbox_inches='tight')
print(f"✓ Saved plot to: {output_dir / 'anomaly_score_distributions.png'}")
plt.show()

# Statistical comparison
print("\nAnomaly Score Statistics by Class (Test Set)")
print("=" * 50)
print(f"Normal (y=0):  Mean = {test_scores[y_test==0].mean():.3f}, Std = {test_scores[y_test==0].std():.3f}")
print(f"Anomaly (y=1): Mean = {test_scores[y_test==1].mean():.3f}, Std = {test_scores[y_test==1].std():.3f}")
print(f"\nMean difference: {test_scores[y_test==1].mean() - test_scores[y_test==0].mean():.3f}")

## Summary

### Key Takeaways:

1. **ROC-AUC** measures overall discriminative ability (higher is better, 0.5 = random)
2. **PR-AUC** is more informative for imbalanced datasets (our case: 89% normal, 11% anomalies)
3. **Threshold selection** depends on operational requirements:
   - Higher threshold → Higher precision, lower recall (fewer false alarms, miss more anomalies)
   - Lower threshold → Lower precision, higher recall (catch more anomalies, more false alarms)

### Performance Interpretation:
- **ROC-AUC > 0.7**: Good separation between classes
- **PR-AUC**: Compare to baseline (random classifier = 0.11 for this dataset)
- **Threshold tuning**: Choose based on cost of false positives vs. false negatives in your use case

### Next Steps:
1. Compare with other anomaly detection methods (LOF, One-Class SVM)
2. Try ontology-based rule validation on detected anomalies
3. Feature importance analysis to understand what drives anomalies
4. Ensemble multiple anomaly detection methods